In [9]:
# install library requests and bs4

import requests, os, bs4
import pandas as pd
import re # for regular expression

# Guide to create executable Python script
# https://martechwithme.com/convert-python-script-app-windows-mac/

# Guide to create scheduler
# https://martechwithme.com/schedule-python-scripts-windows-mac/


# Open local csv file that contain historical data
df_base_usdidr = pd.read_csv('output/Rate.csv', sep = '|')
df_base_usdidr['Tanggal'] = df_base_usdidr['Tanggal'].astype('datetime64[ns]')
df_base_usdidr['Rate'] = df_base_usdidr['Rate'].astype(float)

df_base_usdidr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Tanggal  28 non-null     datetime64[ns]
 1   Rate     28 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 576.0 bytes


In [13]:
# Open link to BI USD IDR website
url = 'https://www.bi.go.id/en/moneter/informasi-kurs/referensi-jisdor/Default.aspx'
# url to get daily BI rates on daily basis
res = requests.get(url)
res.raise_for_status()
BI_usdidr_page = bs4.BeautifulSoup(res.text, 'html.parser')

# using beautiful soup to parse the html content
elems = BI_usdidr_page.select('tr > td')

# Part of Data test to find out content of elems
print(type(elems))
print(len(elems))
print(elems[24].attrs)
print(elems[25].getText())

<class 'bs4.element.ResultSet'>
36
{}

                    14,877.00
                


In [16]:
# Create function to check whether a string is a date, and another function
# to remove any , in a number

# Library to 
import dateutil
from dateutil import parser

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.
    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parser.parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False


def is_angka(string):
    string = string.strip()
    string = string.replace(',','')
    print(string)
    try:
        float(string)
        return True
    except ValueError:
        return False



In [17]:
# Coding to strip all the content of elems and put them to list
content = [x.text.strip() for x in BI_usdidr_page.select('tr > td')]

tanggal = []
rate = []

for i in content:
    if is_angka(i):
        angka = i.strip()
        angka = angka.replace(',','')
        rate.append(angka)
    elif is_date(i):
        dte = dateutil.parser.parse(i)
        tanggal.append(dte)





FOREIGN EXCHANGE REFERENCE RATE
JAKARTA INTERBANK SPOT DOLLAR RATE
USD - IDR​
FOREIGN EXCHANGE REFERENCE RATE
JAKARTA INTERBANK SPOT DOLLAR RATE
USD - IDR​
31 August 2020
14554.00
28 August 2020
14702.00
27 August 2020
14714.00
26 August 2020
14636.00
25 August 2020
14632.00
24 August 2020
14794.00
19 August 2020
14786.00
18 August 2020
14907.00
14 August 2020
14917.00
13 August 2020
14877.00
12 August 2020
14777.00
11 August 2020
14728.00
10 August 2020
14750.00
7 August 2020
14647.00
From

To



In [18]:
#create new df 
df_usdidr = pd.DataFrame({'Tanggal':tanggal,'Rate':rate})
df_usdidr['Rate'] = df_usdidr['Rate'].astype(float)

#append to the database
df_base_usdidr = df_base_usdidr.append(df_usdidr)

#Drop duplicate
df_base_usdidr.drop_duplicates(subset ="Tanggal", 
                     keep = 'first', inplace = True) 

df_base_usdidr.to_csv('output/Rate.csv', index=None, sep = '|')